In [1]:
from fn import op, _, F
from operator import add, mul
from itertools import repeat, chain
from functools import reduce
# from objproxies import LazyProxy
import json
from copy import deepcopy, copy
from pipetools import pipe
from functools import partial
import numpy as np
from datetime import datetime, timedelta
from decimal import Decimal, getcontext, ROUND_DOWN
TWOPLACES = Decimal(10) ** -2       # same as Decimal('0.01')
import pandas as pd
import pprint
# getcontext().prec=None
# getcontext().rounding = ROUND_DOWN

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

def flatmap(f, items):
        return list(map(f, items))
    
def datetime_range(start, end, delta, dt_format='%Y-%m-%d %H:%M:%S'):
    reverse_head = end
    [start, end] = [datetime.strptime(x, dt_format) for x in [start, end]]
    def _datetime_range(start, end, delta):
        current = start
        while current < end:
            yield current
            current += delta
            
    reverse_tail = [dt.strftime(dt_format) for dt in _datetime_range(start, end, delta)]
    return reverse_tail + [reverse_head]
    
def last_index(l):
    return len(l)-1

def retrieve_state(l, offset):
    return l[last_index(l) + offset + 1]

def bound_norm_random(rng, low, high):
    # Add RNG Seed
    res = rng.normal((high+low)/2,(high-low)/6)
    if (res<low or res>high):
        res = bound_norm_random(rng, low, high)
    return Decimal(res)

def env_proc(trigger_step, update_f):
    def env_step_trigger(trigger_step, update_f, step):
        if step == trigger_step:
            return update_f
        else:
            return lambda x: x
    return partial(env_step_trigger, trigger_step, update_f)

def time_step(dt_str, dt_format='%Y-%m-%d %H:%M:%S', days=0, minutes=0, seconds=30):
    dt = datetime.strptime(dt_str, dt_format)
    t = dt + timedelta(days=days, minutes=minutes, seconds=seconds)
    return t.strftime(dt_format)

def ep_time_step(s, dt_str, fromat_str='%Y-%m-%d %H:%M:%S', days=0, minutes=0, seconds=1):
    if s['mech_step'] == 0:
        return time_step(dt_str, fromat_str, days, minutes, seconds)
    else: 
        return dt_str

def round_down(x, fp=TWOPLACES):
    return x.quantize(TWOPLACES, rounding=ROUND_DOWN)

In [3]:
# esceptions:
# point to line giving problems and pass a sentinel

In [4]:
seed = {
    'z': np.random.RandomState(1),
    'a': np.random.RandomState(2),
    'b': np.random.RandomState(3),
    'c': np.random.RandomState(3)
}

# UI Behaviors per Mechanism
def b1m1(step, sL, s):
    return s['s1'] + 1
def b2m1(step, sL, s):
    return s['s1'] + 1

def b1m2(step, sL, s):
    return s['s1'] + 1
def b2m2(step, sL, s):
    return s['s1'] + 1

def b1m3(step, sL, s):
    return s['s1'] + 1
def b2m3(step, sL, s):
    return s['s2'] + 1

# UI Internal States per Mechanism
def s1m1(step, sL, s, _input):
    s['s1'] = s['s1'] + _input
def s2m1(step, sL, s, _input):
    s['s2'] = s['s2'] + _input

def s1m2(step, sL, s, _input):
    s['s1'] = s['s1'] + _input
def s2m2(step, sL, s, _input):
    s['s2'] = s['s2'] + _input
    
def s1m3(step, sL, s, _input):
    s['s1'] = s['s1'] + _input
def s2m3(step, sL, s, _input):
    s['s2'] = s['s2'] + _input

# UI Exogenous States #per Mechanism
proc_one_coef_A = 0.7
proc_one_coef_B = 1.3
def es3p1(step, sL, s, _input):
    s['s3'] = s['s3'] * bound_norm_random(seed['a'], proc_one_coef_A, proc_one_coef_B)
def es4p2(step, sL, s, _input):
    s['s4'] = s['s4'] * bound_norm_random(seed['b'], proc_one_coef_A, proc_one_coef_B)  
def es5p2(step, sL, s, _input):
    s['timestamp'] = ep_time_step(s, s['timestamp'], seconds=1)

def env_a(x): 
    return 10
def env_b(x): 
    return 10
def what_ever(x): 
    return x + 1

# Genesis States    
state_dict = {
    's1': Decimal(0.0),
    's2': Decimal(0.0),
    's3': Decimal(1.0),
    's4': Decimal(1.0),
    'timestamp': '2018-10-01 15:16:24'
}

sim_config = {
    "N": 2
}

exogenous_states = {
    "s3": es3p1,
    "s4": es4p2,
    "timestamp": es5p2
}

env_processes = {
    "s3": env_proc('2018-10-01 15:16:25', env_a),
    "s4": env_proc('2018-10-01 15:16:25', env_b)
}
    
mechanisms = {
    "m1": {
        "behaviors": {
            "b1": b1m1,
            "b2": b2m1
        },
        "states": {
            "s1": s1m1,
            "s2": s2m1,
        }
    },
    "m2": {
        "behaviors": {
            "b1": b1m2,
            "b2": b2m2
        },
        "states": {
            "s1": s1m2,
            "s2": s2m2,
        }
    },
    "m3": {
        "behaviors": {
            "b1": b1m3,
            "b2": b2m3
        },
        "states": {
            "s1": s1m3,
            "s2": s2m3,
        }
    }
}

In [5]:
# if beh list empty, repeat 0 x n_states in list 
def generate_config(mechanisms, exogenous_states):
    es_funcs = [exogenous_states[state] for state in list(exogenous_states.keys())]
    config = list(
        map(
            lambda m: (
                list(mechanisms[m]["states"].values()) + es_funcs,
                list(mechanisms[m]["behaviors"].values())
            ), 
            list(mechanisms.keys())
        )
    )
    return config

# partials = list(map(lambda f: partial(f, step, sL), funcs))
def getColResults(step, sL, s, funcs):
    return list(map(lambda f: f(step, sL, s), funcs))

def getBehaviorInput(step, sL, s, funcs): 
    return op.foldr(_ + _)(getColResults(step, sL, s, funcs))

def apply_env_proc(env_processes, state_dict, step):
    for state in state_dict.keys(): 
        if state in list(env_processes.keys()):
            state_dict[state] = env_processes[state](step)(state_dict[state])
#     return state_dict
            
def mech_step(m_step, sL, state_funcs, behavior_funcs, env_processes, t_step):
    in_copy, mutatable_copy, out_copy = deepcopy(sL), deepcopy(sL), deepcopy(sL)
    last_in_obj, last_mut_obj = in_copy[-1], mutatable_copy[-1]
    
    _input = getBehaviorInput(m_step, sL, last_in_obj, behavior_funcs)

    for f in state_funcs:
        f(m_step, sL, last_mut_obj, _input)
        
    apply_env_proc(env_processes, last_mut_obj, last_mut_obj['timestamp'])
        
    last_mut_obj["mech_step"], last_mut_obj["time_step"] = m_step, t_step
    out_copy.append(last_mut_obj)
    
    del last_in_obj, last_mut_obj, in_copy, mutatable_copy,
    return out_copy

def block_gen(states_list, configs, env_processes, t_step):
    m_step = 0
    states_list_copy = deepcopy(states_list)
    genesis_states = states_list_copy[-1]
    genesis_states['mech_step'], genesis_states['time_step'] = m_step, t_step
    states_list = [genesis_states]
    
    m_step += 1
    for config in configs:
        s_conf, b_conf = config[0], config[1]
        states_list = mech_step(m_step, states_list, s_conf, b_conf, env_processes, t_step)     
        m_step += 1   
        
    t_step += 1
    
    return states_list

def pipeline(states_list, configs, env_processes, time_seq):
    time_seq = [x + 1 for x in time_seq]
    simulation_list = [states_list]
    for time_step in time_seq:
        pipeline_run = block_gen(simulation_list[-1], configs, env_processes, time_step)
        head, *pipeline_run = pipeline_run
        simulation_list.append(pipeline_run)
    
    return simulation_list

def simulation(states_list, configs, env_processes, time_seq, runs):
    pipeline_run = []
    for run in range(runs):
        if run == 0:
            head, *tail = pipeline(states_list, configs, env_processes, time_seq)
            head[-1]['mech_step'], head[-1]['time_step'] = 0, 0
            simulation_list = [head] + tail
            pipeline_run += simulation_list
        else:
            transient_states_list = [pipeline_run[-1][-1]]
            head, *tail = pipeline(transient_states_list, configs, env_processes, time_seq)
            pipeline_run += tail
            
    return pipeline_run

In [6]:
states_list = [state_dict]
configs = generate_config(mechanisms, exogenous_states)
p = pipeline(states_list, configs, env_processes, range(10))
N = sim_config['N']
s = simulation(states_list, configs, env_processes, range(5), N)
# x = flatten(s)
# transient_state = p[-1][1]
# transient_state['mech_step'], transient_state['time_step'] = 0, 0
# head, *tail = pipeline([transient_state], configs, env_processes, range(1))
# head

In [7]:
pd.DataFrame(flatten(s))

,mech_step,s1,s2,s3,s4,time_step,timestamp
0,0,0,0,1,1,0,2018-10-01 15:16:24
1,1,2,2,10,10,1,2018-10-01 15:16:25
2,2,8,8,10,10,1,2018-10-01 15:16:25
3,3,26,26,10,10,1,2018-10-01 15:16:25
4,1,80,80,10.11272650481664925337099703,9.373583088811631158776549455,2,2018-10-01 15:16:26
5,2,242,242,10.48734693326453094234983527,8.620166261663314142236915486,2,2018-10-01 15:16:26
6,3,728,728,11.91324213527073640563759608,6.534876346304093857289840679,2,2018-10-01 15:16:26
7,1,2186,2186,12.51111677744237829906599131,5.931189683155669237263639524,3,2018-10-01 15:16:27
8,2,6560,6560,11.45491115406912332557488657,5.323909548201248804011334243,3,2018-10-01 15:16:27
9,3,19682,19682,11.45492233574810924703732794,5.922305246967391091784578434,3,2018-10-01 15:16:27


In [8]:
# pd.DataFrame(flatten(p))

In [9]:
# flatten(pipeline(states_list, configs, env_processes, range(10)))

In [10]:
states_list = [state_dict]
configs = generate_config(mechanisms, exogenous_states)
# type(pipeline(states_list, configs, env_processes, range(10)))
pd.DataFrame(flatten(pipeline(states_list, configs, env_processes, range(10))))

,mech_step,s1,s2,s3,s4,time_step,timestamp
0,0,0,0,1,1,0,2018-10-01 15:16:24
1,1,2,2,10,10,1,2018-10-01 15:16:25
2,2,8,8,10,10,1,2018-10-01 15:16:25
3,3,26,26,10,10,1,2018-10-01 15:16:25
4,1,80,80,10.32600343338698101192107970,11.77464503140593476970821030,2,2018-10-01 15:16:26
5,2,242,242,8.219165152747849552053712991,10.98017733872146348043050973,2,2018-10-01 15:16:26
6,3,728,728,8.257183331140684599959797063,11.14555732791249226295457795,2,2018-10-01 15:16:26
7,1,2186,2186,7.697614183037259884157485079,11.31602383773220234209463432,3,2018-10-01 15:16:27
8,2,6560,6560,6.589589556253206242453558457,10.11177792894543602078060757,3,2018-10-01 15:16:27
9,3,19682,19682,6.935079384210909962341072506,10.55461981674527424973269651,3,2018-10-01 15:16:27


In [11]:
list(range(2))

[0, 1]

In [12]:
# Decimal('240.0').quantize(Decimal('0.01'), ROUND_DOWN)
# round(Decimal('240.01'), 2)
state_dict = {'s1': Decimal('3323'), 's2': Decimal('2572'), 's3': Decimal('2.657'), 's4': Decimal('7.914'), 'timestamp': '2018-10-01 15:16:26', 'mech_step': 1, 'time_step': 2}
state_dict = {k: v.quantize(TWOPLACES, ROUND_DOWN) for k, v in state_dict.items() if type(v) == Decimal}
state_dict.update(non_numeric)
state_dict

NameError: name 'non_numeric' is not defined

In [ ]:
states_list = [state_dict]
states_list
# {'s1': 8.742503764992283e+170,
#    's2': 1.119720780594405e+171,
#    's3': 4.251811100418906,
#    's4': 6.92728042985628,
#    'timestamp': '2018-10-01 15:16:31',
#    'mech_step': 3,
#    'time_step': 7}
# {'s1': Decimal('2.221384236472411425810660220E+330'),
#    's2': Decimal('2.845096899831187764579913612E+330'),
#    's3': Decimal('3.622519693611211445932083740'),
#    's4': Decimal('5.161487081026327858663750787'),
#    'timestamp': '2018-10-01 15:16:32',
#    'mech_step': 3,
#    'time_step': 8}

In [ ]:
from decimal import Decimal, MAX_EMAX
# Decimal(2.5347751615574597e+167)**2
MAX_EMAX**5

In [ ]:
1000 * 1000

In [ ]:
from datetime import datetime, timedelta
# from datetime import timedelta  
# st = datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
st = datetime.now()
print(st)
x = st + timedelta(seconds=60)
print(x)

In [ ]:
s = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
datetime_object = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
datetime_object

In [ ]:
datetime.fromtimestamp()

In [ ]:
from datetime import datetime, timedelta

# def time_step_func(year, month, day, hour, minute, second):
a = datetime(2016, 11, 15, 9, 59, 25)#.strftime("%Y-%m-%d %H:%M:%S")
b = a + timedelta(seconds=60) # days, seconds, then other fields.
print(a)
print(b)
st = datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
def time_step_func(dt_str, fromat_str='%Y-%m-%d %H:%M:%S', days=0, minutes=0, seconds=1):
    dt = datetime.strptime(dt_str, fromat_str)
    t = dt + timedelta(days=days, minutes=minutes, seconds=seconds)
    return t.strftime(fromat_str)

time_step_func('2018-10-01 15:16:24')

In [ ]:
def time_step_func(x, y):
    return x + y

In [ ]:
for m in M: #M is the parameter sweep array
    # for Pairwise, run below separately
    for n in N: #N is the total number of Montecarlo runs
        for t in T: #T[len(T)]-T[0] is the duration of each simulation
            # Pipeline
            for b in Behaviors:
                for M in Mechanisms:
                    for s in States:
                        update(s)

In [ ]:
[{'s1': 2}] + [{'s1': 3}]
list(zip([block_gen] * 2, range(2)))
len([0,1])

In [ ]:
states_list = [state_dict]
configs = generate_config(mechanisms, exogenous_states)
def step_func(x, y):
    return x + y
pipeline(states_list, configs, env_processes, step_func)

In [ ]:
# def s3step1(step, sL, s, _input):
#     s['s3'] = s['s3']    
# # def s3step2(step, sL, s, _input):
#     s['s3'] = s['s3'] + 1
# def s3step3(step, sL, s, _input):
#     s['s3'] = s['s3'] * bound_norm_random(proc_one_coef_A, proc_one_coef_B)

In [ ]:
# 15 + 28.933333333333334 + 4

In [ ]:
# def multiply(x,y,z):
#         return x * y * z

# # create a new function that multiplies by 2
# dbl = partial(partial(multiply,2), multiply, 2)
# print(dbl(2))